In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P5', 'P6', 'P7', 'P8', 'P9']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    #!rm autoclass.r-params
    #!cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/Hp2.csv', index = True, header=True)

2020-04-27 16:25:18 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P5.tsv' as 'real location' with error 0.01
2020-04-27 16:25:18 INFO     Detected encoding: ascii
2020-04-27 16:25:18 INFO     Found 3200 rows and 2 columns
2020-04-27 16:25:18 DEBUG    Checking column names
2020-04-27 16:25:18 DEBUG    Index name 'index'
2020-04-27 16:25:18 DEBUG    Column name 'P5'
2020-04-27 16:25:18 INFO     Checking data format
2020-04-27 16:25:18 INFO     Column 'P5'
2020-04-27 16:25:18 INFO     count    3200.000000
2020-04-27 16:25:18 INFO     mean       -0.083701
2020-04-27 16:25:18 INFO     std         2.579438
2020-04-27 16:25:18 INFO     min       -42.765333
2020-04-27 16:25:18 INFO     50%        -0.276978
2020-04-27 16:25:18 INFO     max        42.745123
2020-04-27 16:25:18 INFO     ---
2020-04-27 16:25:18 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv' as 'real location' with error 

Time: 420 sec.

2020-04-27 16:32:19 INFO     Extracting autoclass results
2020-04-27 16:32:19 INFO     Found 3200 cases classified in 26 classes
2020-04-27 16:32:23 INFO     Aggregating input data
2020-04-27 16:32:23 INFO     Writing classes + probabilities .tsv file
2020-04-27 16:32:24 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P6.tsv' as 'real location' with error 0.01
2020-04-27 16:32:24 INFO     Detected encoding: ascii
2020-04-27 16:32:24 INFO     Found 3200 rows and 2 columns
2020-04-27 16:32:24 DEBUG    Checking column names
2020-04-27 16:32:24 DEBUG    Index name 'index'
2020-04-27 16:32:24 DEBUG    Column name 'P6'
2020-04-27 16:32:24 INFO     Checking data format
2020-04-27 16:32:24 INFO     Column 'P6'
2020-04-27 16:32:24 INFO     count    3200.000000
2020-04-27 16:32:24 INFO     mean       -0.287767
2020-04-27 16:32:24 INFO     std         2.171565
2020-04-27 16:32:24 INFO     min       -36.499477
2020-04-27 16:32:24 INFO     50%      

[0.11353711790393013, 0.07268170426065163, 0.09302325581395349, 0.12301587301587301, 0.09022556390977443, 0.09401709401709402, 0.2610619469026549, 0.09644670050761421, 0.2, 0.13274336283185842, 0.12698412698412698, 1.0, 0.11594202898550725, 0.16901408450704225, 0.9782608695652174, 0.22641509433962265, 0.46511627906976744, 0.09090909090909091, 0.10416666666666667, 0.1282051282051282, 0.19047619047619047, 0.3, 0.125, 0.5833333333333334, 0.21428571428571427, 0.36363636363636365]
['DROPBOX', 'CLOUDFLARE', 'SPOTIFY', 'APPLE_ICLOUD', 'DNS', 'HTTP_CONNECT', 'CONTENT_FLASH', 'HTTP', 'AMAZON', 'INSTAGRAM', 'MS_ONE_DRIVE', 'IP_ICMP', 'MS_ONE_DRIVE', 'APPLE_ITUNES', 'DNS', 'WHATSAPP', 'OFFICE_365', 'WIKIPEDIA', 'FACEBOOK', 'GMAIL', 'HTTP_PROXY', 'HTTP', 'HTTP', 'DNS', 'GOOGLE', 'WINDOWS_UPDATE']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
0.24840375346642982


2020-04-27 16:32:24 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:32:24 INFO     Writing run file
2020-04-27 16:32:24 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:32:24 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:32:24 INFO     Running clustering...


...

[0.08395061728395062, 0.085, 0.10614525139664804, 0.09731543624161074, 0.07885304659498207, 0.10185185185185185, 0.24401913875598086, 0.12, 0.15428571428571428, 0.1, 0.1206896551724138, 0.09433962264150944, 1.0, 0.2391304347826087, 0.19642857142857142, 0.2835820895522388, 0.2391304347826087, 0.1346153846153846, 1.0, 0.3684210526315789, 0.125, 0.2, 0.35294117647058826]
['CLOUDFLARE', 'APPLE', 'WHATSAPP', 'CLOUDFLARE', 'HTTP_CONNECT', 'DROPBOX', 'CONTENT_FLASH', 'SPOTIFY', 'AMAZON', 'WINDOWS_UPDATE', 'HTTP', 'EBAY', 'IP_ICMP', 'DROPBOX', 'HTTP_CONNECT', 'OFFICE_365', 'DNS', 'SSL', 'DNS', 'FACEBOOK', 'HTTP', 'GOOGLE', 'INSTAGRAM']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
0.24024780341253224


2020-04-27 16:55:25 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:55:25 INFO     Writing run file
2020-04-27 16:55:25 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:55:25 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:55:25 INFO     Running clustering...


Time: 528 sec.

2020-04-27 17:04:13 INFO     Extracting autoclass results
2020-04-27 17:04:13 INFO     Found 3200 cases classified in 28 classes
2020-04-27 17:04:16 INFO     Aggregating input data
2020-04-27 17:04:16 INFO     Writing classes + probabilities .tsv file


[0.07668711656441718, 0.08264462809917356, 0.09859154929577464, 0.07114624505928854, 0.07111111111111111, 0.08866995073891626, 0.08796296296296297, 0.15723270440251572, 0.13043478260869565, 0.20382165605095542, 0.19333333333333333, 0.3013698630136986, 1.0, 0.1388888888888889, 0.11764705882352941, 0.14634146341463414, 0.1388888888888889, 0.09090909090909091, 0.2777777777777778, 1.0, 0.17777777777777778, 0.1, 0.2, 0.2926829268292683, 0.2, 0.16129032258064516, 0.47058823529411764, 0.4]
['CLOUDFLARE', 'HTTP_CONNECT', 'HTTP_PROXY', 'INSTAGRAM', 'CLOUDFLARE', 'HTTP', 'HTTP_CONNECT', 'AMAZON', 'APPLE', 'DROPBOX', 'OFFICE_365', 'CONTENT_FLASH', 'IP_ICMP', 'DNS', 'INSTAGRAM', 'AMAZON', 'APPLE_ITUNES', 'EBAY', 'HTTP_PROXY', 'DNS', 'HTTP', 'HTTP_CONNECT', 'INSTAGRAM', 'SPOTIFY', 'FACEBOOK', 'GOOGLE', 'DROPBOX', 'APPLE_ICLOUD']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
0.23127851194376653
<class 'pandas.core.frame.DataFrame'>
RangeIndex